# Урок 8. Снижение размерности данных

#### Загрузка библиотке

In [31]:
import numpy as np
from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import warnings
warnings.simplefilter('ignore')

#### Глобальные функции

In [13]:
# Функция определения главных компонент через SVD

def pca_with_svd(X, n_components):
    U, s, Vt = np.linalg.svd(X)
    
    D = np.zeros_like(X, dtype=float)
    D[np.diag_indices(min(X.shape))] = s
    V = Vt.T
    
    # проверка
    np.testing.assert_array_almost_equal(U.dot(D).dot(Vt), X)
    
    W = V[:, :n_components]
    Z = X.dot(W)
    
    return Z

In [24]:
# Функция для вычислении нормы Фробениуса

def frobenius(X):
    return np.sum(X ** 2) ** 0.5

In [25]:
# Функция для определения доли вероятности

def accuracy_score(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

## 1. Можно ли отобрать наиболее значимые признаки с помощью PCA?

Нет, нельзя, т.к. метод главных компонент задействует все исходные признак, создавая через линейную комбинацию новые.

## 2*. Примите участие в одном или двух соревнованиях и пришлите

псевдоним: WhiteAndBlack Fox

## 3*. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [3]:
X, y = load_iris(return_X_y=True)

In [16]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
max_component = 9
print('Норма Фробениуса')
print(f" * Исходная матрица: {frobenius(X_scaled)}") 
for i in range(1, max_component):
    X_pca = pca_with_svd(X_scaled, n_components=i)
    print(f" * Матрица при ГК = {i} матрица X: {frobenius(X_pca)}")

Норма Фробениуса
 * Исходная матрица: 24.494897427831784
 * Матрица при ГК = 1 матрица X: 20.92306556123646
 * Матрица при ГК = 2 матрица X: 23.97663953101038
 * Матрица при ГК = 3 матрица X: 24.431389124151476
 * Матрица при ГК = 4 матрица X: 24.494897427831777
 * Матрица при ГК = 5 матрица X: 24.494897427831777
 * Матрица при ГК = 6 матрица X: 24.494897427831777
 * Матрица при ГК = 7 матрица X: 24.494897427831777
 * Матрица при ГК = 8 матрица X: 24.494897427831777


К сожалению, при применении нормы Форбениуса, при увеличении главного компонента, нора увеличивается, причем заметно.

## 4*. Обучить любую модель классификации на датасете iris до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [27]:
X, y = load_iris(return_X_y=True)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 5)

In [29]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Обучаем модель до применения PCA

In [35]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

95.23809523809523 93.33333333333333


#### Обучем модель после применения PCA

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size = 0.3, random_state = 5)

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

91.42857142857143 88.88888888888889
